# AI Pair Programming

Dans ce notebook nous allons demander à une intelligence artificielle de générer du code Python pour réaliser des tâches de traitement automatique de corpus.

Avant de commencer, choisissez un outil comme [Bard](https://bard.google.com/u/2/chat) ou [ChatGPT](https://chat.openai.com/) et créez un compte.

Vous pouvez ensuite demander à l'outil de créer du code. Avant de commencer, n'hésitez pas à lire [cet article](https://exocoding.com/ai-code-generation/) qui détaille les **bonnes pratiques** pour créer des _prompts_ efficaces dans le cadre de la génération de code par l'intelligence artificielle.


### 1. Algorithme simple en Python

Demandez à l'IA de générer un code python qui lance un décompte du réveillon du Nouvel An. Le code doit imprimer les nombres de 10 à 0 avec un intervalle d'une seconde, puis imprimer "Bonne année" à la fin.

In [ ]:
# Votre code ici

### 2. Détection du sujet de la phrase

Demandez à l'IA d'extraire le sujet dans une phrase.
Demandez ensuite de générer le code Python qui réalise cette tâche et testez le ci-dessous.

In [ ]:
# Votre code ici


### 3. Entités nommmées liées à Wikidata

Demandez à l'IA d'extraire les entités nommées d'un texte en français, et de les lier à un identifiant wikidata.
Demandez ensuite à l'IA de générer le code Python pour réaliser cette tâche et testez le ci-dessous.

In [1]:
import spacy
import requests

# Charger le modèle français de spaCy
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    # Si le modèle n'est pas installé, indiquer l'installation
    raise RuntimeError("Le modèle spaCy 'fr_core_news_sm' n'est pas installé. Installez-le via `python -m spacy download fr_core_news_sm`.")

def link_wikidata(entity_label: str, language: str = "fr", limit: int = 3):
    """
    Interroge l'API Wikidata pour retrouver des éléments correspondant à `entity_label`.
    Retourne une liste de (Q-ID, label, description) possible.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "language": language,
        "search": entity_label,
        "format": "json",
        "limit": limit
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    results = []
    for entry in data.get("search", []):
        qid = entry.get("id")
        label = entry.get("label")
        desc = entry.get("description")
        results.append((qid, label, desc))
    return results

def extract_and_link(text: str):
    """
    Extrait les entités nommées d'un texte, puis les lie à des identifiants Wikidata.
    Retourne un dictionnaire : entité → liste de propositions (Q-ID, label, description).
    """
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        label = ent.text
        # Éviter les doublons (mappage simple)
        if label not in entities:
            # Lier via Wikidata
            linked = link_wikidata(label)
            entities[label] = linked
    return entities

if __name__ == "__main__":
    texte = ("Paris est la capitale de la France où le Président Macron habite. "
             "En tant que chef de l'État français, il convie souvent son homologue belge, "
             "le roi Philippe, et sa femme Mathilde, à un goûter dans le jardin du Palais de l'Elysée.")
    mapping = extract_and_link(texte)
    # Afficher les résultats
    for ent, candidates in mapping.items():
        print(f"Entité détectée : '{ent}'")
        for qid, label, desc in candidates:
            print(f"  → {qid}: {label} — {desc}")
        print()

RuntimeError: Le modèle spaCy 'fr_core_news_sm' n'est pas installé. Installez-le via `python -m spacy download fr_core_news_sm`.

### 4. A vous de jouer

Pensez à une analyse que vous voudriez faire sur un texte. Demandez à l'IA de générer un code python qui réalise cette analyse et testez le ci-dessous

In [3]:
import re

def sentiment_fallback_fr(text: str):
    """
    Analyse de sentiment simple en français basée sur un mini-lexique.
    Renvoie : score, details
    """
    positive_words = [
        "extraordinaires", "soutiennent", "émancipation",
        "prospérité", "beau", "écoute", "motivent", "encouragent"
    ]
    negative_words = [
        "manquent", "difficiles", "préjugés", "malheur",
        "supprimer", "fragiliser", "détresse", "licenciement",
        "explosé", "ravagé", "manque", "insuffit", "tension"
    ]

    text_lower = text.lower()
    pos = sum(text_lower.count(w) for w in positive_words)
    neg = sum(text_lower.count(w) for w in negative_words)

    if pos > neg:
        sentiment = "positif"
    elif neg > pos:
        sentiment = "négatif"
    else:
        sentiment = "neutre"

    return sentiment, {"positifs": pos, "negatifs": neg}

def sentiment_analysis(text: str):
    """
    Tente d'abord Transformers pour l'analyse multilingue.
    Si indisponible, bascule automatiquement sur la méthode lexicale.
    """
    try:
        from transformers import pipeline
        classifier = pipeline(
            "sentiment-analysis",
            model="nlptown/bert-base-multilingual-uncased-sentiment"
        )
        # Modèles limitent parfois la longueur → découpage simple
        chunks = [text[i:i+500] for i in range(0, len(text), 500)]
        results = classifier(chunks)
        # Moyenne naïve des scores
        avg_score = sum((i+1) * (r['label'][0]) for i, r in enumerate(results))
        return results
    except Exception:
        return sentiment_fallback_fr(text)

# TEST SUR TON TEXTE -----------------------------------------------

full_text = """(…L’action sociale existe en Belgique depuis plus d’un siècle. En 1925, une loi fusionne les hospices civils et les bureaux de bienfaisance communaux en une institution unique. Plus de cinquante ans plus tard, en 1976, la loi organique des CPAS consacre les centres publics de l’aide sociale. Celle-ci est garantie à chaque citoyen. Aujourd’hui, en 2025, face au train de réformes prévues par le Ministre wallon des pouvoirs locaux, nous, directeurs généraux des CPAS wallons, réaffirmons à quel point notre beau métier peut encore évoluer.

Les travailleurs des CPAS ne sont pas des variables à ajuster
Nous sommes 261 directeurs généraux de CPAS en Wallonie. A ce titre, nous sommes (entre autres) les chefs du personnel de plus de 20.000 travailleurs : assistants sociaux, éducateurs, aides familiales, conseillers en insertion professionnelle, informateurs sociaux, gestionnaires administratifs, médiateurs de dettes, puéricultrices, agents d’entretien…

Des équipes que nous encourageons et motivons en permanence. Parce que le métier est dur, parce que les CPAS manquent de moyens et les travailleurs sociaux de considération. Parce que l’air du temps clive ceux qui travaillent et les autres. Et cette tendance est malheureusement exponentielle… Pourtant, ces femmes et ces hommes accompagnent, écoutent, soignent, hébergent, insèrent, réparent.

Les CPAS rendent des services extraordinaires à la collectivité. Ils accueillent vos parents dans des maisons de repos, vos enfants dans des crèches, soutiennent des milliers de personnes dans la recherche d’un emploi ou d’un logement. Et oui, les CPAS aident financièrement les personnes qui traversent un accident de vie, un licenciement, une maladie. Parce que leur salaire ne suffit plus, parce qu’elles sont dans des situations d’immense détresse. Avec un objectif clair : l’émancipation. Avec une certitude : ça peut arriver à tout le monde.

Quand rien ne va plus, les CPAS répondent présents
Les CPAS ont toujours gardé le cap. Quand la terre entière était en stand-by à cause du covid ; quand les factures d’énergie ont explosé ; quand les inondations ont ravagé les habitations. Les équipes ont toujours été sur le pont. Malgré des conditions de travail difficiles et des préjugés souvent tenaces. Les CPAS sont le dernier filet de la sécurité sociale. Ils tiennent toujours – mais jusqu’à quand ?

Jusqu’en 2030, si on en croit le projet de réforme du ministre concernant « la fusion des communes et des CPAS en une entité unique. » C’était l’objet d’une réunion à laquelle le ministre a convié mardi les bourgmestres et présidents de CPAS. Comme nous ne voulons pas ajouter du malheur au monde, nommons bien les choses : intégrer les CPAS aux communes revient tout bonnement à les supprimer et ce, sans analyse préalable des bénéfices de cette mesure. A diminuer de facto la protection sociale des citoyens. A mettre au même niveau l’action sociale et la rénovation d’une voirie.

 À lire aussi Les communes, « cochons payeurs » du système belge
Pour l’efficience, contre la méthode
Nous, les 261 directeurs généraux des CPAS wallons, voulons le redire clairement ici : nous souhaitons que les CPAS évoluent. Nous sommes pour l’efficience et l’évolution de nos métiers dans une société en perpétuel changement.

Mais nous tenons également à le réaffirmer fermement : l’action sociale ne peut pas et ne pourra jamais être une variable d’ajustement budgétaire.

Or, les réformes en cours (exclusion du chômage, réforme de la fonction publique, suppression des CPAS) en prennent le chemin.

Toutes impacteront l’action sociale. Elles vont fragiliser les CPAS, qui devront faire des choix. De nombreux programmes subsidiés ont déjà pris fin dans tout le pays (aide au logement comme Housing First, lutte contre la précarité des jeunes…).

Les services dits « facultatifs » comme les repas à domicile ou les structures d’hébergement, deviennent déjà une marge de manœuvre budgétaire. Minimisés ou carrément supprimés, ils sont actuellement analysés par les pouvoirs locaux, contraints de mettre la main au portefeuille pour compenser l’arrêt des subventions.

Quant à la réforme du chômage, elle risque de pousser près de 30.000 Wallons vers nos CPAS. Soyons clairs : il s’agit d’un défi que nous pouvons et allons assumer, comme tous les autres

– pour autant qu’on nous en donne les moyens financiers, humains et matériels.

– pour autant que nous soyons étroitement associés aux réformes. Avec une réelle concertation et la faculté pour les équipes de terrain de coconstruire l’avenir avec le ministre et son cabinet.

– Pour autant que ces réformes soient élaborées dans une sécurité juridique absolue, et assorties d’une justification financière quant à leur bien-fondé. Or, nous entendons, dans le cadre de la suppression des CPAS prévue en 2030, que la Wallonie pourrait déjà permettre aux communes volontaires d’intégrer les CPAS dans un système hybride, sans tenir compte des lois fédérales actuelles. Et que les gains annoncés sont de l’ordre de l’hypothèse.

À lire aussi L’IA dans les CPAS belges : anticiper la tempête sociale qui s’annonce
Entendus par les autorités, nous serons les partenaires du changement.

Nous connaissons les difficultés financières de notre Région. Mais le manque d’argent n’empêche nullement la concertation, la coconstruction et la réelle prise en compte de nos métiers et in fine, des citoyens.

Tout comme le manque de moyens n’empêche pas le respect de la législation : nous demandons au gouvernement de respecter les articles 1 et 2 de la Loi Organique des CPAS qui garantissent le droit de toute personne à l’aide sociale et qui attribuent un CPAS à chaque commune du Royaume.

Nous sommes convaincus que la prospérité d’un pays se mesure à la manière dont il protège les plus fragiles. C’est pour ces raisons, pour nos équipes, pour les citoyens de Wallonie, que nous comptons sur l’écoute attentive et la prise en compte de nos préoccupations par le gouvernement wallon.

Nous demandons à être considérés comme de vrais partenaires loyaux. Pour des CPAS autonomes, garants de la légalité, de l’équité et de la dignité pour tous les citoyens.

)"""

result = sentiment_analysis(full_text)
result

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


('négatif', {'positifs': 7, 'negatifs': 14})


### Pour aller plus loin...

En tant qu'étudiant ULB vous avez accès gratuitement au [Github student pack](https://education.github.com/pack). Ce pack vous donne permet d'utiliser [Github copilot](https://github.com/features/copilot), un outil d'auto-complétion de code grâce à l'intelligence artificielle. Ceci peut être très utile si vous voulez réaliser des tâches complexes sans être un expert en python.